In [1]:
from transformers import AutoModelForSeq2SeqLM

model_name = "UBC-NLP/AraT5v2-base-1024"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

num_params = sum(p.numel() for p in model.parameters())
print(f"Nombre total de paramètres : {num_params / 1e6:.2f} millions")


config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

2025-05-22 18:07:57.655742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747937277.915094      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747937277.997679      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Nombre total de paramètres : 367.51 millions


In [2]:
import pandas as pd

df = pd.read_parquet("/kaggle/input/data-set-full")

# Affiche les premières lignes pour vérifier
print(df.head())

from datasets import Dataset

dataset = Dataset.from_pandas(df)


  index                                            summary  \
0     1  بدأت محكمة ميونخ النظر في اتهامات متعلقة برجل ...   
1     2  أعلن حاكم كارولاينا الشمالية حالة الطورائ في م...   
2     3  قالت وزارة الدفاع الأمريكية (البنتاغون) إن داع...   
3     4  "فشلت الولايات المتحدة وروسيا في الاتفاق على ك...   
4     5  أثنى مدرب بايرن ميونيخ كارلو أنشيلوتي على مساع...   

                                             article  text_length  \
0  "بدأت اليوم الجمعة( 23 أيلول/ سبتمبر 2016 ) في...          318   
1  "قال مسؤولون إن شخصا أصيب إصابة بالغة بعد تعرض...          379   
2  "أعلن مسؤول في البنتاغون أن جنودا أمريكيين في ...          201   
3  "اجتمعت المجموعة الدولية لدعم سوريا على هامش ا...          194   
4  "قال الإيطالي كارلو أنشيلوتي المدير الفني لفري...          183   

   summary_length  diff diff_category  \
0              37   281      فرق كبير   
1              33   346      فرق كبير   
2              35   166      فرق كبير   
3              28   166      فرق كبير   
4      

In [3]:
from transformers import AutoTokenizer

model_name = "UBC-NLP/AraT5v2-base-1024"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_input_length = 512   # ou la longueur max de ton texte
max_target_length = 128  # ou la longueur max de ton résumé

def preprocess_function(examples):
    inputs = examples["text_clean"]
    targets = examples["summary"]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="max_length")

    labels_ids = labels["input_ids"]
    # Remplacer les pads par -100 pour ignorer la perte sur ces tokens
    labels_ids = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels_ids
    ]

    model_inputs["labels"] = labels_ids
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


 

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/48937 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]


In [4]:
from transformers import AutoModelForSeq2SeqLM

model_name = "UBC-NLP/AraT5v2-base-1024"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [5]:
!pip install huggingface_hub transformers datasets

from huggingface_hub import notebook_login
notebook_login()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

In [6]:
!pip install evaluate
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8c1098535be5c01de9bc1a67014faf2f2004497e7c54b9070df12f1a8ad53ea3
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [7]:
import evaluate
import numpy as np

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")  # BLEU classique, utile si tu fais de la traduction ou du résumé mot à mot


In [8]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Supprimer les valeurs -100 dans les labels (elles sont ignorées pendant l'entraînement)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Décodage des prédictions et des labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Suppression des espaces superflus
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Calcul des métriques
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # Retourne les scores d'intérêt
    return {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "bleu": bleu_result["bleu"],
    }


In [9]:
!wandb login --relogin 845f8dedf69c8bf78f12b093e8a4ff627911b2e8



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [10]:
import wandb
wandb.init(
    project="arabic-summarization",  # Choisis un nom de projet
    name="arabart-run-last"             # Nom du run (facultatif)
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mohamedmourinou2018 (mohamedmourinou2018-ensa-al-hoceima) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [13]:
max_train_size = 20000

# Prendre seulement les 20k premiers exemples dans train
train_dataset = split_dataset["train"].select(range(min(len(split_dataset["train"]), max_train_size)))

# Pour la validation, tu peux garder tout ou aussi limiter si tu veux
val_dataset = split_dataset["test"]  # ou limiter aussi si besoin


In [20]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=500,
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    max_steps=1000,
    logging_steps=10,
    fp16=True,
    push_to_hub=True,                # <-- Active le push automatique
    hub_model_id="mohamedmou/arabic-summarizer",  
    report_to="wandb",  # important !
# <-- Nom du repo HuggingFace Hub
# si tu as une GPU compatible
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # 🔥 Ajout ici

)


/tmp/ipykernel_35/2417636065.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [21]:
trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint_path = "/kaggle/working/results/checkpoint-500"

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

model.push_to_hub("mohamedmou/arabic-summarizer", commit_message="Push checkpoint 500")
tokenizer.push_to_hub("mohamedmou/arabic-summarizer")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mohamedmou/arabic-summarizer/commit/235eb247c4bd6b59916e6d4f3e7853b92f1d86d8', commit_message='Upload tokenizer', commit_description='', oid='235eb247c4bd6b59916e6d4f3e7853b92f1d86d8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mohamedmou/arabic-summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='mohamedmou/arabic-summarizer'), pr_revision=None, pr_num=None)

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Nom du modèle Hugging Face
model_name = "mohamedmou/arabic-summarizer"

# Charger le tokenizer et le modèle depuis Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Exemple de texte arabe à résumer
text = """
"تشهد منطقة الشرق الأوسط وشمال أفريقيا تحديات اقتصادية وسياسية كبيرة بسبب الصراعات المستمرة والتغيرات الجيوسياسية في المنطقة. رغم ذلك، تبرز فرص للنمو الاقتصادي من خلال تطوير قطاعات جديدة مثل التكنولوجيا والطاقة المتجددة. الحكومات في بعض الدول تسعى إلى تعزيز الاستقرار وتحسين البنية التحتية لدعم التنمية المستدامة. كما أن الاستثمار في التعليم والشباب يعد مفتاحاً لمستقبل أفضل. ومع ذلك، لا تزال التحديات الأمنية والاجتماعية تشكل عائقاً أمام تحقيق هذه الأهداف، مما يتطلب تعاوناً إقليمياً ودولياً مكثفاً للتغلب عليها."



"""

# Tokenizer le texte, limiter la longueur max à 524 tokens (ajuste selon ton modèle)
inputs = tokenizer(text, return_tensors="pt", max_length=524, truncation=True)

# Générer un résumé avec beam search (4 faisceaux), limiter à 100 tokens max
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=100,
    num_beams=4,
    early_stopping=True
)

# Décoder la sortie en texte lisible (sans tokens spéciaux)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Résumé généré :", summary)


Résumé généré : تشهد منطقة الشرق الأوسط وشمال أفريقيا تحديات اقتصادية وسياسية كبيرة بسبب الصراعات المستمرة والتغيرات الجيوسياسية. ومع ذلك، تبرز فرص للنمو الاقتصادي من خلال تطوير قطاعات جديدة مثل التكنولوجيا والطاقة المتجددة.


In [32]:
!pip install gradio


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 82.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.7 MB/s eta 0:00:00


In [33]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "mohamedmou/arabic-summarizer"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

iface = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(lines=10, placeholder="Insérez ici un texte arabe à résumer...", label="Texte à résumer"),
    outputs=gr.Textbox(lines=5, label="Résumé généré"),
    title="Résumé automatique - Arabe",
    description="Entrez un texte en arabe pour générer un résumé automatique."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://48eadfacdb89fe2419.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
